导入包  
TensorFlow Datasets 是 TensorFlow 生态系统中的一个核心组件，它旨在简化数据获取和预处理流程，为机器学习和深度学习项目提供即用型数据集。TFDS 提供了一系列高质量、标准化的数据集，支持多种数据类型，包括图像、文本、视频等，下面是其主要特点和使用方法的概览：

In [1]:
import keras.callbacks
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers

In [11]:
EPOCHS = 10  # 训练轮数
BATCH_SIZE = 128  # 批处理大小
VERBOSE = 1  # 是否打印日志
OPTIMIZER = optimizers.Adam()  # 优化器
VALIDATION_SPLIT = 0.95  # 是指训练集的划分比例，0.95表示训练集95%，验证集5%

In [3]:
IMG_ROWS, IMG_COLS = 28, 28  # 输入图片的维度
INPUT_SHAPE = (IMG_ROWS, IMG_COLS, 1)
NB_CLASSES = 10  # 输出的类别数

定义LeNET网络

In [4]:
def build_model(input_shape, classes):
    model = models.Sequential()
    # CONV => RELU => POOL
    model.add(layers.Convolution2D(20 ,(5, 5), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # CONV => RELU => POOL
    model.add(layers.Convolution2D(50, (5, 5), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # Flatten => RELU layers
    model.add(layers.Flatten())
    model.add(layers.Dense(500, activation='relu'))
    # a softmax classifier
    model.add(layers.Dense(classes, activation='softmax'))
    return model

### `build_model` 函数解析

该函数运用 Keras（TensorFlow 库的构成部分）构建了一个序列化的卷积神经网络（CNN）模型，广泛应用于图像分类任务。下文详述模型结构：

- **输入层**  
  通过`input_shape`参数隐式设定，确保与输入图像尺寸（不含颜色通道）相符。

- **卷积层 1**  
  添加了20个5x5大小的滤波器，每个滤波器后接ReLU激活函数，负责提取图像基础特征。

- **最大池化 1**  
  采用2x2大小的最大池化减少空间复杂度，增进特征的平移不变性。

- **卷积层 2**  
  增加至50个5x5滤波器，深化特征学习，提取更高级特征。

- **最大池化 2**  
  继续通过2x2最大池化降低维度，简化数据结构。

- **展平层**  
  将卷积层输出的二维特征图转化为一维向量，适配全连接层。

- **全连接层（Dense 1）**  
  含500个神经元，使用ReLU激活，整合并强化特征。

- **输出层（Dense 2）**  
  神经元数等于分类数，采用softmax激活，输出各类别概率，实现多分类。

- **模型概览**  
  显示模型结构摘要，包括参数总量，帮助评估模型复杂度。

- **返回模型**  
  构建完毕的模型待编译与训练。

此架构作为图像分类的基本CNN模型，为众多项目奠定基础。然而，现代实践倾向于加入批量归一化、dropout机制或Inception模块等，以提升性能和模型的泛化能力。


加载数据集

In [5]:
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
# 重塑
X_train = x_train.reshape(x_train.shape[0], IMG_ROWS, IMG_COLS, 1)
X_test = x_test.reshape(x_test.shape[0], IMG_ROWS, IMG_COLS, 1)
# 归一化
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0
# 
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# 转换为one-hot编码
Y_train = tf.keras.utils.to_categorical(y_train, NB_CLASSES)
Y_test = tf.keras.utils.to_categorical(y_test, NB_CLASSES)

In [6]:
model = build_model(input_shape=INPUT_SHAPE, classes=NB_CLASSES)
model.compile(loss='categorical_crossentropy',
              optimizer=OPTIMIZER,
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 20)        520       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 20)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 50)          25050     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 4, 50)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 800)               0         
                                                                 
 dense (Dense)               (None, 500)               4

In [8]:
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./LeNet_logs')
]

In [12]:
history = model.fit(X_train, Y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    verbose=VERBOSE,
                    validation_split=VALIDATION_SPLIT,
                    callbacks=callbacks)
score = model.evaluate(X_test, Y_test, verbose=VERBOSE)

Epoch 1/10
24/24 [==============================] - 5s 224ms/step - loss: 0.0992 - accuracy: 0.9743 - val_loss: 0.1596 - val_accuracy: 0.9507
Epoch 2/10
24/24 [==============================] - 4s 190ms/step - loss: 0.0777 - accuracy: 0.9783 - val_loss: 0.1462 - val_accuracy: 0.9547
Epoch 3/10
24/24 [==============================] - 4s 187ms/step - loss: 0.0690 - accuracy: 0.9833 - val_loss: 0.1376 - val_accuracy: 0.9580
Epoch 4/10
24/24 [==============================] - 4s 189ms/step - loss: 0.0549 - accuracy: 0.9850 - val_loss: 0.1347 - val_accuracy: 0.9587
Epoch 5/10
24/24 [==============================] - 5s 193ms/step - loss: 0.0417 - accuracy: 0.9893 - val_loss: 0.1197 - val_accuracy: 0.9631
Epoch 6/10
24/24 [==============================] - 4s 190ms/step - loss: 0.0368 - accuracy: 0.9903 - val_loss: 0.1327 - val_accuracy: 0.9588
Epoch 7/10
24/24 [==============================] - 4s 187ms/step - loss: 0.0232 - accuracy: 0.9957 - val_loss: 0.1231 - val_accuracy: 0.9640
Epoch 

In [13]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.09191161394119263
Test accuracy: 0.972000002861023
